In [1]:
import json, re, os, sys
import pandas as pd
from os import path as osp
import openai
from tqdm import tqdm
from configs.openai_key import peiqi_key

In [2]:
openai.api_base = "https://api.chatanywhere.cn/v1"
openai.api_key = peiqi_key

In [3]:
# 得到小规模样本的id并保存
from tools.extract_sample_ids import get_sampled_ids
sample_ids = get_sampled_ids('baichuan-oreilly-sample',
                                 'baichuan-13b-chat')
with open("/mnt/mfs/opsgpt/evaluation/ops-cert-eval/subset_ids.json", 'w') as f:
    json.dump(list(sample_ids), f, indent=4)

In [4]:
# 获得题目
with open("/mnt/mfs/opsgpt/evaluation/ops-cert-eval/v2/all.json") as f:
    data = json.load(f)
sample_data = [
    q for q in data if q['id'] in sample_ids
]

In [27]:
sample_data[0]

{'id': '9781119862642-A0003',
 'type': 0,
 'question': 'You are purchasing a mobile device that allows you to use multi-finger gestures to interact with the device. What type of touchscreen technology does this device most likely use?',
 'choices': ['Capacitive', 'Infrared', 'SAW', 'Resistive'],
 'answer': 'A',
 'solution': "Capacitive touchscreens are a little less accurate than resistive touchscreens but more responsive. They respond to changes in electrical current, and as such, the human fingertip works great as the facilitator of input. Resistive screens require a heavier touch and don't support multi-touch input. Infrared screens don't work with multi-touch gestures, but like resistive screens, they can be used with gloves on. Surface Acoustic Wave (SAW) is another type of touchscreen, but it is not popular",
 'topic': ['Mobile Devices'],
 'source': '9781119862642/c01.xhtml'}

In [14]:
model_list = [
    "text-curie-001",
    "text-babbage-001",
    "text-ada-001",
    "davinci"
]

In [5]:
_hint = """
你是一个英译汉的翻译机器人。你需要将用户发来的选择题翻译成中文，你的回答只能有三行，第一行是题面的翻译，第二行是选项的翻译，第三行是类别的翻译，其他不要给出任何多余信息，下面是一个答案的样例：
---
您购买的移动设备允许您使用多指手势与设备交互。该设备最有可能使用什么类型的触摸屏技术？
['电容式', '红外线', 'SAW', '电阻式']
移动设备
---
"""

def get_translation(problem, choices, topic):
    prompt = f"{problem}\n{choices}\n{topic}"
    messages = [
        {'role': 'system', 'content': _hint},
        {'role': 'user', 'content': prompt}
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", 
        messages=messages, 
        temperature=0)
    answer = response['choices'][0]['message']['content']
    return answer

In [7]:
import json
from tqdm import tqdm
failed = []
translated = []
finished = []
with open("/mnt/mfs/opsgpt/evaluation/translated/sample.json", 'r') as f:
    data = json.load(f)
finished = set([d['id'] for d in data])
print(len(finished))
idx = 0
for q in tqdm(sample_data):
    if q['id'] in finished:
        continue
    try:
        ans = get_translation(q['question'], q['choices'], q['topic'])
        question, choices, topic = ans.split('\n')
        q['question'] = question
        q['choices'] = eval(choices)
        q['topic'] = topic
        translated.append(q)
        if idx % 100 == 0:
            with open("/mnt/mfs/opsgpt/evaluation/translated/sample.json", 'w') as f:
                json.dump(translated, f)
        idx += 1
    except:
        # print(q['id'], "failed!")
        failed.append(q['id'])

4201


  0%|          | 15/4954 [00:36<3:47:53,  2.77s/it]